In [109]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import math
import numpy as np

In [2]:
os.getcwd()

'/private/var/tmp'

In [3]:
os.chdir('/Users/artemkovtunenko/Downloads')

In [247]:
df_NOVA=pd.read_csv("df_NOVA_Status_Severity_Filtered.csv").dropna(axis=0, how='any')
print(len(df_NOVA))
train_NOVA = df_NOVA.sample(frac=0.5)
print(len(train_NOVA))
test_NOVA = df_NOVA.loc[~df_NOVA.index.isin(train_NOVA.index)]
print(len(test_NOVA))

348
174
174


In [248]:

train_data = train_NOVA[['field_words_bag']]
train_label = train_NOVA[['Classification']]
test_data = test_NOVA[['field_words_bag']]
test_label = test_NOVA[['Classification']]

In [249]:
clf = tree.DecisionTreeClassifier()

In [250]:
type(train_data)


pandas.core.frame.DataFrame

In [251]:

train_data.dropna(axis=0, how='any')
train_label.dropna(axis=0, how='any')
test_data.dropna(axis=0, how='any')
test_label.dropna(axis=0, how='any')

train_data.dropna(axis=1, how='any')
train_label.dropna(axis=1, how='any')
test_data.dropna(axis=1, how='any')
test_label.dropna(axis=1, how='any')

train_data_np = train_data.as_matrix()
train_label_np = train_label.as_matrix()
test_data_np = test_data.as_matrix()
test_label_np = test_label.as_matrix()

print(type(train_data_np))

<class 'numpy.ndarray'>


In [252]:
train_data_np[0]


array([ '[different, process, neutron, send, exception, exception, handle, message, relay, correctly, example, follow, command, fail, restriction, interface, attach, expect, result, expect, command, fail, message, follow, port, belong, subnets, across, enter, prise, allow, actual, result, however, instead, message, relay, back, clientexception, server, either, incapable, perform, request, operation, http, request, follow, info, oslo, message, drivers, impl, rabbit, admin, admin, connect, amqp, server, admin, admin, catch, remote, badrequest, request, port, belong, subnets, across, enter, prise, allow, traceback, recent, call, last, local, python, dist, package, oslo, message, dispatcher, dispatch, reply, executor, callback, local, python, dist, package, oslo, message, dispatcher, dispatch, executor, callback, local, python, dist, ...]'], dtype=object)

In [253]:
#_clf = clf.fit(train_data_np, train_label_np)

In [254]:
train_label_np[0]

array(['Whitebox Testing'], dtype=object)

In [255]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])

In [256]:
classifier.fit(train_data_np.ravel(), train_label_np)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...lti_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1))])

In [257]:
print(train_data.index[2], train_label.index[2])
for i,j in (zip(train_data, train_label)):
    print(i,j)


114 114
field_words_bag Classification


In [258]:
list_test_data = []
list_test_label = []
for i in test_data.index:
    list_test_data.append(test_data['field_words_bag'][i])
for i in test_label.index:
    
    list_test_label.append(test_label['Classification'][i])


In [259]:
predicted = classifier.predict(list_test_data)

In [260]:
predicted = classifier.predict(list_test_data)

In [261]:
for i in range(len(predicted)):
    print (str(list_test_data[i][:40])+' => '+str(predicted[i]))

[cinder, volumes, stick, state, attach,  => Volume unusable
[step, reproduce, boot, shutoff, migrate => Whitebox Testing
[version, ubuntu, cloud, kilo, issue, ba => Whitebox Testing
[today, look, like, fail, miss, ephemera => Design Limitation
[python, noarch, common, noarch, python, => Oversize
[occur, versions, cinder, instal, bulk,  => Inconsistent state
[user, reclaim, interval, anything, user => Resource Allocation
[node, create, containers, work, fine, r => Whitebox Testing
[summary, fail, attach, encrypt, create, => Compute Migration
[look, briefly, code, drivers, report, x => Compute Migration
[migration, block, migration, float, rou => Compute Migration
[simple, setup, total, three, host, cont => Design Limitation
[version, commit, create, boot, prior, d => Compute Migration
[current, master, branch, backend, step, => Inconsistent state
[post, copy, live, migration, cause, los => Compute Migration
[several, change, community, newton, neu => Design Limitation
[come, newton, sum

In [262]:
print(classification_report(test_label, predicted))

                                precision    recall  f1-score   support

Backward/Forward Compatibility       0.00      0.00      0.00         5
             Compute Migration       0.29      0.56      0.38        16
             Design Limitation       0.05      0.04      0.04        23
            Inconsistent state       0.13      0.22      0.16        18
      Incorrect Error Handling       0.15      0.12      0.13        17
              Negative Testing       0.00      0.00      0.00         3
                      Oversize       0.00      0.00      0.00         4
                Race Condition       0.33      0.12      0.18         8
                  Reboot Cycle       0.00      0.00      0.00         3
           Resource Allocation       0.17      0.09      0.12        11
            SIG_TERM/Core Dump       0.00      0.00      0.00         3
               Security Threat       0.00      0.00      0.00         2
             Storage Migration       0.00      0.00      0.00  

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
